# Chapter 05 이미지 처리 능력이 탁월한 CNN

## 5.2 CNN 모델 구현하기

In [ ]:
# 모듈 불러오기
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [ ]:
# gpu 사용가능 여부 확인
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
USE_CUDA

True

In [ ]:
# 하이퍼파라미터 설정
EPOCHS = 40
BATCH_SIZE = 64

In [ ]:
# 데이터로더 정의
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=True,
                          download=True,
                          transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                          train=False,
                          download=True,
                          transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
                          ])),
    batch_size=BATCH_SIZE, shuffle=True)

현재 만드는 CNN의 커널은 5x5, 컨볼루션 계층은 2개

[Conv2d 층 설명](https://gaussian37.github.io/dl-pytorch-conv2d/)

In [ ]:
# CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.drop = nn.Dropout2d()

        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.drop(self.conv2(x)), 2))

        x = x.view(-1, 320)

        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return x

In [ ]:
# model과 최적화함수
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
# 훈련 함수
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)} ({100.*batch_idx/len(train_loader):.0f}%)]\tLoss:{loss.item():.6f}')

In [ ]:
len(train_loader), len(train_loader.dataset)

(938, 60000)

In [ ]:
# 평가 함수
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

모델 실행

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f'[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%')

Train Epoch: 1 [0/60000 (0%)]	Loss:2.313305
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.370155
Train Epoch: 1 [25600/60000 (43%)]	Loss:1.088256
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.860652
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.747864
[1] Test Loss: 0.6642, Accuracy: 74.47%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.963815
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.785075
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.856021
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.665301
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.767851
[2] Test Loss: 0.5618, Accuracy: 78.50%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.534215
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.672763
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.747651
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.554629
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.652202
[3] Test Loss: 0.5137, Accuracy: 80.29%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.810447
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.802413
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.581880
Train Epoch: 4 [38

### 전체코드 및 연습

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
EPOCHS = 50  # Horizon 추가해서  10에폭 늘림
BATCH_SIZE = 64

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
        train=True,
        download=True,
        transform=transforms.Compose([
            transforms.RandomHorizontalFlip(),        # 추가함. 데이터가 늘어남에 따라 에폭 수 늘려야될것으로 보임
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
        train=False,
        download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=BATCH_SIZE, shuffle=True
)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return x

In [ ]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(model, train_loader, optimizer, epoch):
    print(f"Train Epoch : {epoch}")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        # if batch_idx % 200 == 0:
        #     print(f"[{batch_idx*len(data)}/{len(train_loader.dataset)} ({100.*batch_idx/len(train_loader):.0f}%)]\tLoss: {loss.item:.6f}")

In [ ]:
def evaluate(mode, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.*correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
from tqdm import tqdm

for epoch in tqdm(range(1, EPOCHS + 1)):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f"[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

  0%|          | 0/50 [00:00<?, ?it/s]

Train Epoch : 1


  2%|▏         | 1/50 [00:22<18:07, 22.20s/it]

[1] Test Loss: 0.6868, Accuracy: 74.17%
Train Epoch : 2


  4%|▍         | 2/50 [00:43<17:34, 21.96s/it]

...

[48] Test Loss: 0.3310, Accuracy: 87.61%
Train Epoch : 49


 98%|█████████▊| 49/50 [18:01<00:22, 22.07s/it]

[49] Test Loss: 0.3311, Accuracy: 87.59%
Train Epoch : 50


100%|██████████| 50/50 [18:23<00:00, 22.06s/it]

[50] Test Loss: 0.3305, Accuracy: 87.59%


성능이 더 안좋아짐

## 5.3 ResNet으로 컬러 데이터셋에 적용하기

In [1]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# 컬러 데이터셋은 복잡하므로 에폭 수가 더 필요함
EPOCHS = 300
BATCH_SIZE = 128

### 5.3.2 CIFAR-10 데이터셋
32x32 크기의 이미지 6만개 포함<br/>
label은 10가지 동물

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=True,
                     download=True,
                     transform=transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                             (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                     train=False,
                     download=True,
                     transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5),
                                             (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./.data/cifar-10-python.tar.gz to ./.data
Files already downloaded and verified


## 5.3.3 CNN을 깊게 쌓는 방법

* 중간중간에 입력을 더해 데이터 소실을 막아 깊게 쌓을 수 있음.
* 계층이 깊어짐에 따라 신호의 강도가 감소하는데, 입력 데이터를 몇 계층씩 건너뛰어 출력에 더함으로써 이 현상을 완화
* 배치정규화를 통해 드롭아웃과 같은 효과를 냄

In [5]:
# Basic Block 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        # 두 번째 블록부터 입력값을 더해주는 모듈
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

main 모델 정의

In [6]:
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

### 모델의 흐름
입력이 들어오면 일반적인 방식과 같이 컨볼루션, 배치 정규화, 활성화 함수를 차례로 통과하고, <br/>
BasicBlock 층을 갖고 있는 layer1, layer2, layer3를 통과함.<br/>
이렇게 나온 값에 평균 풀링을 하고 마지막 계층을 거쳐 분류 결과를 출력

학습 효율을 높이기 위해 학습률 감소기법 사용

In [7]:
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50,
                                      gamma=0.1)

In [8]:
# 모델의 전체 계층
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [9]:
# 학습 및 테스트 코드

def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 가장 높은 값을 가진 인덱스가 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS + 1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)

    print(f"[{epoch}] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1] Test Loss: 1.5474, Accuracy: 46.91%
[2] Test Loss: 1.2882, Accuracy: 56.73%
[3] Test Loss: 1.2935, Accuracy: 57.96%
[4] Test Loss: 0.9893, Accuracy: 67.19%
[5] Test Loss: 1.0351, Accuracy: 66.32%
[6] Test Loss: 0.8056, Accuracy: 73.14%
[7] Test Loss: 0.7427, Accuracy: 74.95%
[8] Test Loss: 0.9487, Accuracy: 68.34%
[9] Test Loss: 0.8356, Accuracy: 72.49%
[10] Test Loss: 0.7906, Accuracy: 73.32%
[11] Test Loss: 0.7640, Accuracy: 75.10%
[12] Test Loss: 0.9492, Accuracy: 69.88%
[13] Test Loss: 0.8032, Accuracy: 74.38%
[14] Test Loss: 0.6962, Accuracy: 76.36%
[15] Test Loss: 0.8175, Accuracy: 73.15%
[16] Test Loss: 0.5723, Accuracy: 80.47%
[17] Test Loss: 0.6353, Accuracy: 78.57%
[18] Test Loss: 0.8825, Accuracy: 70.60%
[19] Test Loss: 0.8103, Accuracy: 74.42%
[20] Test Loss: 0.6250, Accuracy: 79.04%
[21] Test Loss: 0.7970, Accuracy: 74.39%
[22] Test Loss: 0.6367, Accuracy: 78.77%
[23] Test Loss: 0.7103, Accuracy: 77.11%
[24] Test Loss: 0.9351, Accuracy: 71.55%
[25] Test Loss: 0.7574, A

KeyboardInterrupt: ignored